In [10]:
import pandas as pd
df = pd.read_json("datosParsed3.json")

In [11]:
df

,id,text,label
0,aaexyuw,\n571 Main Page\n\n\nComputer Science 571\nCON...,course
1,abbdqt,\nObject-Oriented Programming (Graduate) Home ...,course
2,achmly,\n\n\nECE/CS 752 Spring 1996\n\n\n\n\nECE/CS 7...,course
3,aciio,"Last-Modified: Thursday, 01-Feb-96 22:23:44 GM...",course
4,ackfxrep,\n\nEECS401 Web Page for Fall '96\n\n\nWelcome...,course
...,...,...,...
6618,zxgxje,\n\nHomePage of Daqing Li\n\n\n\n\nWelcome to ...,student
6619,zyidaxg,"Last-Modified: Sunday, 25-Aug-96 22:35:51 GMT\...",student
6620,zyrphu,\n\n\n\nHOME \n\n\n\n\n \n \n \n\n\nMarla Bak...,student
6621,zyvupbc,\n\nPatrice Caire\n\n\n\n\nPatrice Caire \nVir...,student


In [7]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
model_name = 'doc2query/all-with_prefix-t5-base-v1'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
prefix = "answer2question"
outputColumn = []
cont = 0
for i in df['text']:
    text = prefix+": "+ i
    input_ids = tokenizer.encode(text, max_length=384, truncation=True, return_tensors='pt')
    outputs = model.generate(
        input_ids=input_ids,
        max_length=64,
        do_sample=True,
        top_p=0.95,
        num_return_sequences=20)

    outputText = i

    for i in range(len(outputs)):
        query = tokenizer.decode(outputs[i], skip_special_tokens=True)
        outputText = outputText + ' ' +  query
    outputColumn.append(outputText)
    if cont % 100 == 0:
        print(cont)
        print(cont/df.size)
    cont += 1
df['expandedText'] = outputColumn

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


0
0.0
100
0.005032965926820675
200
0.01006593185364135
300
0.015098897780462027
400
0.0201318637072827
500
0.025164829634103377
600
0.030197795560924053
700
0.035230761487744726
800
0.0402637274145654
900
0.04529669334138608
1000
0.050329659268206754
1100
0.05536262519502743
1200
0.060395591121848106
1300
0.06542855704866878
1400
0.07046152297548945
1500
0.07549448890231013
1600
0.0805274548291308
1700
0.08556042075595148
1800
0.09059338668277216
1900
0.09562635260959283
2000
0.10065931853641351
2100
0.10569228446323418
2200
0.11072525039005486
2300
0.11575821631687554
2400
0.12079118224369621
2500
0.12582414817051688
2600
0.13085711409733755
2700
0.13589008002415823
2800
0.1409230459509789
2900
0.14595601187779958
3000
0.15098897780462026
3100
0.15602194373144093
3200
0.1610549096582616
3300
0.16608787558508228
3400
0.17112084151190296
3500
0.17615380743872364
3600
0.1811867733655443
3700
0.186219739292365
3800
0.19125270521918566
3900
0.19628567114600634
4000
0.20131863707282702
4100

In [21]:
df['expandedText'] = outputColumn
df = df[['id', 'expandedText', 'label']]

In [24]:
df.iloc[0,1]

'\n571 Main Page\n\n\nComputer Science 571\nCONCEPTS IN PROGRAMMING LANGUAGES\n\n\nINSTRUCTOR\nEllis Horowitz\nComputer Science Dept.\nUniversity of Southern California\nLos Angeles, Ca. 90089\nOFFICE: 300 SAL, PHONE: 213-740-8056\nEMAIL: horowitz@pollux.usc.edu\nOffice hrs:Mon-Wed: 10:30-Noon, Tues: 1:00-3:00pm\n\n\n Textbooks\n Course Overview\n Week-by-Week Topics\n Introduction\n Survey Form\n NTU Syllabus\n Programming Language References\n\nClassnotes\n571 Classnotes\nLex&Yacc Notes\nLanguages Directory\nSample Tests\nREADME.571\nNew JavaScript Classnotes\nNew Java Classnotes\nhttp://www.usc.edu/dept/cs/class_info/571/books.html http://www.usc.edu/dept/cs/class_info/571/overview.html http://www.usc.edu/dept/cs/class_info/571/topics.html http://www.usc.edu/dept/cs/class_info/571/intro.html http://www.usc.edu/dept/cs/class_info/571/form.html http://www.usc.edu/dept/cs/class_info/571/ntu.html http://sunset.usc.edu/~horowitz/Homework/hw2.html ftp://ftp.usc.edu/pub/csci571/571slides.t

In [25]:
df.to_json('expanded_data.json')

In [16]:
df.head

<bound method NDFrame.head of             id  text    label
0      aaexyuw  None   course
1       abbdqt  None   course
2       achmly  None   course
3        aciio  None   course
4     ackfxrep  None   course
...        ...   ...      ...
6618    zxgxje  None  student
6619   zyidaxg  None  student
6620    zyrphu  None  student
6621   zyvupbc  None  student
6622   zywpsym  None  student

[6623 rows x 3 columns]>

In [3]:
for i in df['text']:
    print(i)
    break

Date: Wed, 20 Nov 1996 23:30:42 GMT
Server: NCSA/1.5.2
Last-modified: Wed, 16 Oct 1996 04:38:33 GMT
Content-type: text/html
Content-length: 1361

571 Main Page


Computer Science 571
CONCEPTS IN PROGRAMMING LANGUAGES


INSTRUCTOR
Ellis Horowitz
Computer Science Dept.
University of Southern California
Los Angeles, Ca. 90089
OFFICE: 300 SAL, PHONE: 213-740-8056
EMAIL: horowitz@pollux.usc.edu
Office hrs:Mon-Wed: 10:30-Noon, Tues: 1:00-3:00pm


 Textbooks
 Course Overview
 Week-by-Week Topics
 Introduction
 Survey Form
 NTU Syllabus
 Programming Language References

Classnotes
571 Classnotes
Lex&Yacc Notes
Languages Directory
Sample Tests
README.571
New JavaScript Classnotes
New Java Classnotes

